# 1. Explore

## Run name

In [1]:
import time

project_name = 'dog_breed'
step_name = 'Explore'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: dog_breed_Explore_20190420_114642


## Important params

## Import pkgs

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.display import display

import os
import gc
import zipfile
import h5py
import pickle
import math
from PIL import Image
import shutil

from tqdm import tqdm
from multiprocessing import cpu_count

from sklearn.metrics import confusion_matrix, accuracy_score

cpu_amount = cpu_count()
print('cpu_amount: ', cpu_amount)

cpu_amount:  4


## Project folders

In [3]:
cwd = os.getcwd()
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
feature_folder = os.path.join(cwd, 'feature')
post_pca_feature_folder = os.path.join(cwd, 'post_pca_feature')
log_folder = os.path.join(cwd, 'log')
print('input_folder: \t\t\t%s' % input_folder)
print('output_folder: \t\t\t%s' % output_folder)
print('model_folder: \t\t\t%s' % model_folder)
print('feature_folder: \t\t%s' % feature_folder)
print('post_pca_feature_folder: \t%s' % post_pca_feature_folder)
print('log_folder: \t\t\t%s' % log_folder)

train_folder = os.path.join(input_folder, 'data_train')
val_folder = os.path.join(input_folder, 'data_val')
test_folder = os.path.join(input_folder, 'data_test')

if not os.path.exists(post_pca_feature_folder):
    os.mkdir(post_pca_feature_folder)
    print('Create folder: %s' % post_pca_feature_folder)

labels_csv_file = os.path.join(input_folder, 'labels.csv')
sample_submission_csv_file = os.path.join(input_folder, 'sample_submission.csv')
print('\nlabels_csv_file: \t\t%s' % labels_csv_file)
print('sample_submission_csv_file: \t%s' % sample_submission_csv_file)

input_folder: 			D:\Kaggle\dog-breed-identification\input
output_folder: 			D:\Kaggle\dog-breed-identification\output
model_folder: 			D:\Kaggle\dog-breed-identification\model
feature_folder: 		D:\Kaggle\dog-breed-identification\feature
post_pca_feature_folder: 	D:\Kaggle\dog-breed-identification\post_pca_feature
log_folder: 			D:\Kaggle\dog-breed-identification\log

labels_csv_file: 		D:\Kaggle\dog-breed-identification\input\labels.csv
sample_submission_csv_file: 	D:\Kaggle\dog-breed-identification\input\sample_submission.csv


## Preview data

In [4]:
labels_csv = pd.read_csv(labels_csv_file)
print('labels_csv.shape is {0}.'.format(labels_csv.shape))
display(labels_csv.head(2))

labels_csv.shape is (10222, 2).


,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo


In [5]:
from collections import Counter
count_cat = Counter(labels_csv['breed'])
data = {
    'breed': list(count_cat.keys()),
    'count': list(count_cat.values())
}
count_cat_pd = pd.DataFrame(data=data)
display(count_cat_pd.head())
print('max value: %s' % count_cat_pd['count'].max())

,breed,count
0,boston_bull,87
1,dingo,80
2,pekinese,75
3,bluetick,85
4,golden_retriever,67


max value: 126


In [6]:
# 我需要重写ImageGenerator
# 需要有batch_size生成能力，暂时不需要多核并行能力
# rotation, shift, shear, zoom, channel_shift, brightness，调用默认的实现看行不行
# 我需要一张图片，然后转成np.array

In [11]:
breed_folder = os.path.join(train_folder, os.listdir(train_folder)[0])
img_name = os.listdir(breed_folder)[1]
img_file = os.path.join(breed_folder, img_name)
print(img_file)

D:\Kaggle\dog-breed-identification\input\data_train\affenpinscher\01268f0007876a692907bda89468184c.jpg


In [12]:
from keras.preprocessing.image import load_img

pil_img = load_img(img_file)
print(type(pil_img))

<class 'PIL.JpegImagePlugin.JpegImageFile'>
